In [ ]:
import os
import cv2
import json
import numpy as np
import open3d as o3d
from scipy.spatial.transform import Rotation

np.set_printoptions(precision=5, suppress=True, linewidth=500)

In [ ]:
def npy2ply(input_path, output_path):
    points = np.load(input_path).astype(np.float32)*1e-3
    rot_mat = Rotation.from_euler('xyz', [6, 0, 0], degrees=True).as_matrix()
    new_p = points @ rot_mat
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(new_p)
    o3d.visualization.draw_geometries([pcd])
    o3d.io.write_point_cloud(output_path, pcd, write_ascii=True)

ip = "/home/tatp/ws/graphics/yolov7/runs/detect/kinect/points/color_000.npy"
npy2ply(ip, ip.replace(".npy", ".ply"))

In [ ]:
input_path
points = np.load(input_path).astype(np.float32)*1e-3


In [ ]:
obj = "jimu"
data_folder = f"/home/tatp/ws/GreatWall/DLabSim/data/object/{obj}/3"
points_dir = os.path.join(data_folder, "points")
imgs_dir = os.path.join(data_folder, "images")

points = {}
for p in os.listdir(points_dir):
    points[p.split(".")[0]] = np.load(os.path.join(points_dir, p)).astype(np.float32)*1e-3

with open(os.path.join(data_folder, "images.json")) as fp:
    cam_ext = json.load(fp)

rot_mat = Rotation.from_euler('xyz', [6, 0, 0], degrees=True).as_matrix()
new_points = []
for i, ext in enumerate(cam_ext):
    tmat = np.eye(4)
    tmat[:3,3] = cam_ext[ext]['position']
    tmat[:3,:3] = Rotation.from_quat(cam_ext[ext]['orientation']).as_matrix()
    print(tmat)
    new_p = (points[ext] @ rot_mat + np.array([-0.04, 0.0175, 0])) @ tmat[:3,:3].T + tmat[:3,3]
    # new_p = (points[ext] * 0.9 @ rot_mat) @ tmat[:3,:3].T + tmat[:3,3]
    # new_p = (points[ext] @ rot_mat) @ tmat[:3,:3].T + tmat[:3,3]
    new_p_c = new_p[np.linalg.norm(new_p, axis=1) < 0.15]
    new_points.append(new_p_c)

pcds = []
for i, p in enumerate(new_points):
    # if i in {0,1}:
    #     continue
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(p)
    pcds.append(pcd)
# o3d.visualization.draw_geometries(pcds + [o3d.geometry.TriangleMesh.create_coordinate_frame()])
o3d.visualization.draw_geometries(pcds)

combined_pcd = o3d.geometry.PointCloud()
for pcd in pcds:
    combined_pcd += pcd

o3d.io.write_point_cloud(os.path.join(data_folder, "combined.ply"), combined_pcd, write_ascii=True)


In [ ]:
points

In [ ]:
# new_p = (points[ext] @ rot_mat + np.array([-0.03, 0, 0])) @ tmat[:3,:3].T + tmat[:3,3]
ps = points[ext][177133:177135]
print(ps)
print(rot_mat)
print(ps @ rot_mat)
print(ps @ rot_mat + np.array([-0.03, 0, 0]))

# ps

In [ ]:
for c in cam_ext:
    tmat = np.eye(4)
    tmat[:3,3] = cam_ext[c]['position']
    tmat[:3,:3] = Rotation.from_quat(cam_ext[c]['orientation']).as_matrix()
    print(tmat[:3,3])
    print(np.linalg.norm(tmat[:2,3]))

In [ ]:
np.argmax(points[ext], axis=0)

In [ ]:
os.listdir(points_dir)

In [ ]:
for p in new_points:
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(p)
    o3d.visualization.draw_geometries([pcd, o3d.geometry.TriangleMesh.create_coordinate_frame()])


In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(new_points[0])
o3d.visualization.draw_geometries([pcd])
# o3d.visualization.draw_geometries([pcd, o3d.geometry.TriangleMesh.create_coordinate_frame()])


In [ ]:
import cv2

# Create a VideoCapture object
cap = cv2.VideoCapture(5)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error opening video capture")
else:
    # Read and display video frames until the user presses 'q'
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()

        # If frame is read correctly, ret is True
        if ret:
            # Display the frame
            cv2.imshow('Camera', frame)

        # Break the loop if the user presses 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture object and close the window
    cap.release()
    cv2.destroyAllWindows()
